In [ ]:
!pip install datasets
import datasets
dataset  = datasets.load_dataset('hyperpartisan_news_detection', 'bypublisher')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 362 kB 31.3 MB/s 
     |████████████████████████████████| 1.1 MB 58.3 MB/s 
     |████████████████████████████████| 101 kB 12.8 MB/s 
     |████████████████████████████████| 140 kB 58.3 MB/s 
     |████████████████████████████████| 212 kB 71.0 MB/s 
     |████████████████████████████████| 596 kB 71.2 MB/s 
     |████████████████████████████████| 127 kB 71.6 MB/s 
     |████████████████████████████████| 144 kB 73.5 MB/s 
     |████████████████████████████████| 94 kB 4.1 MB/s 
     |████████████████████████████████| 271 kB 11.5 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
ERROR: pip's de

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/600000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/600000 [00:00<?, ? examples/s]

Dataset hyperpartisan_news_detection downloaded and prepared to /root/.cache/huggingface/datasets/hyperpartisan_news_detection/bypublisher/1.0.0/7f4215b0474950ddf516e806400ab81d098b3da3b3a919a13cd1a4cf2c677012. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.4 MB 28.9 MB/s 
     |████████████████████████████████| 6.6 MB 52.5 MB/s 


In [ ]:
train_size = 5000
val_size = 1000
import numpy as np
train_indices = np.random.randint(0, len(dataset['train']), train_size)
val_indices = np.random.randint(0, len(dataset['validation']), val_size)
train_dataset = dataset['train'].select(train_indices)
val_dataset = dataset['validation'].select(val_indices)

In [ ]:
import pandas as pd

In [ ]:
df_train = pd.DataFrame(train_dataset)
df_test = pd.DataFrame(val_dataset)

In [ ]:
from transformers import AutoTokenizer,TFBertModel
tokenizer = AutoTokenizer.from_pretrained('bigscience/bloom-1b3')
bert = TFBertModel.from_pretrained('bigscience/bloom-1b3', from_pt=True)

You are using a model of type bloom to instantiate a model of type bert. This is not supported for all configurations of models and can yield errors.


Downloading:   0%|          | 0.00/3.21G [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['h.10.mlp.dense_4h_to_h.bias', 'h.20.mlp.dense_h_to_4h.bias', 'h.20.self_attention.query_key_value.bias', 'h.2.mlp.dense_4h_to_h.bias', 'h.11.input_layernorm.weight', 'h.19.self_attention.query_key_value.weight', 'h.13.post_attention_layernorm.bias', 'h.5.post_attention_layernorm.weight', 'h.12.post_attention_layernorm.bias', 'h.12.mlp.dense_4h_to_h.bias', 'h.4.post_attention_layernorm.weight', 'h.23.mlp.dense_4h_to_h.weight', 'h.14.self_attention.dense.weight', 'h.22.mlp.dense_h_to_4h.weight', 'h.2.mlp.dense_h_to_4h.weight', 'h.11.self_attention.query_key_value.bias', 'h.16.mlp.dense_h_to_4h.weight', 'h.21.mlp.dense_h_to_4h.bias', 'h.22.input_layernorm.weight', 'h.15.self_attention.query_key_value.bias', 'h.22.self_attention.query_key_value.bias', 'h.18.mlp.dense_4h_to_h.bias', 'h.8.self_attention.dense.weight', 'h.21.post_attention_layernorm.bias', 'h.4.post_attention_layernorm.bias', 'h.

In [ ]:
x_train = tokenizer(
    text=df_train.text.tolist(),
    add_special_tokens=True,
    max_length=1024,
    truncation=True,
    padding=True, 
    return_tensors='tf',
    return_token_type_ids = False,
    return_attention_mask = True,
    verbose = True)
x_test = tokenizer(
    text=df_test.text.tolist(),
    add_special_tokens=True,
    max_length=1024,
    truncation=True,
    padding=True, 
    return_tensors='tf',
    return_token_type_ids = False,
    return_attention_mask = True,
    verbose = True)

In [ ]:
input_ids = (x_train['input_ids'])
attention_mask = (x_train['attention_mask'])

In [ ]:
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.initializers import TruncatedNormal
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.metrics import CategoricalAccuracy
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Input, Dense

In [ ]:
max_len = 1024
input_ids = Input(shape=(max_len,), dtype=tf.int32, name="input_ids")
input_mask = Input(shape=(max_len,), dtype=tf.int32, name="attention_mask")
embeddings = bert(input_ids,attention_mask = input_mask)[0]
out = tf.keras.layers.GlobalMaxPool1D()(embeddings)
out = Dense(128, activation='relu')(out)
out = tf.keras.layers.Dropout(0.1)(out)
out = Dense(32,activation = 'relu')(out)
y = Dense(5,activation = 'sigmoid')(out)
model = tf.keras.Model(inputs=[input_ids, input_mask], outputs=y)
model.layers[2].trainable = True

In [ ]:
optimizer = Adam(
    learning_rate=5e-05, # this learning rate is for bert model , taken from huggingface website 
    epsilon=1e-08,
    decay=0.01,
    clipnorm=1.0)
# Set loss and metrics
loss =CategoricalCrossentropy(from_logits = True)
metric = CategoricalAccuracy('balanced_accuracy'),
# Compile the model
model.compile(
    optimizer = optimizer,
    loss = loss, 
    metrics = metric)

In [ ]:
df_train

,text,title,hyperpartisan,url,published_at,bias
0,<p>Apple (NASDAQ:AAPL) told shareholders Frida...,Apple to Shareholders: Vote Against Icahn Buyb...,True,http://foxbusiness.com/features/2013/12/27/app...,2016-01-29,0
1,<p>0758 GMT - Sterling trades weaker before th...,Sterling Weaker Before U.K. Labor Market Surve...,True,http://foxbusiness.com/features/2017/08/16/ste...,2017-08-16,0
2,<p /> \n\n<p>Europe's largest twin-engined pas...,Airbus A350-1000 stages maiden flight,True,http://foxbusiness.com/features/2016/11/24/air...,2016-11-24,0
3,<p>.......... .......... .......... .............,‘I wasn’t being brave. I was saving my baby.’,False,https://abqjournal.com/940874/i-wasnt-being-br...,2017-02-01,2
4,<p>Venice.</p> \n\n<p>A few moments after Ital...,How Venice is Dying,True,https://counterpunch.org/2006/07/14/how-venice...,2006-07-14,4
...,...,...,...,...,...,...
4995,<p /> \n\n<p>Investors were getting indigestio...,2 Reasons to Worry About Jack in the Box Inc.,True,http://foxbusiness.com/markets/2016/02/23/2-re...,2016-03-27,0
4996,<p /> \n\n<p>Photo by Marcel Trindade | <a hre...,"Mad Dog, Meet Eris, Queen of Strife",True,https://counterpunch.org/2017/03/28/mad-dog-me...,2017-03-28,4
4997,<p>&amp;#160;</p> \n\n<p>Shank: With similar n...,An Interview with Noam Chomsky,True,https://counterpunch.org/2007/02/22/an-intervi...,2007-02-22,4
4998,<p>Nothing scares feminists more than powerful...,Liberal Vice Magazine’s Feminist Page Slut-Sha...,True,https://dailywire.com/news/14699/liberal-vice-...,2017-03-22,0


In [ ]:
y_train = to_categorical(df_train.bias)
y_test = to_categorical(df_test.bias)

In [ ]:
train_history = model.fit(
    x ={'input_ids':x_train['input_ids'],'attention_mask':x_train['attention_mask']} ,
    y = y_train,
    validation_data = (
    {'input_ids':x_test['input_ids'],'attention_mask':x_test['attention_mask']}, y_test
    ),
  epochs=5,
  batch_size=2
)

Epoch 1/5


/usr/local/lib/python3.7/dist-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


2500/2500 [==============================] - 2209s 878ms/step - loss: 1.5262 - balanced_accuracy: 0.2914 - val_loss: 1.5048 - val_balanced_accuracy: 0.2920
Epoch 2/5
2500/2500 [==============================] - 2164s 866ms/step - loss: 1.5063 - balanced_accuracy: 0.3068 - val_loss: 1.5017 - val_balanced_accuracy: 0.2920
Epoch 3/5
2500/2500 [==============================] - 2165s 866ms/step - loss: 1.5066 - balanced_accuracy: 0.3006 - val_loss: 1.5027 - val_balanced_accuracy: 0.2920
Epoch 4/5
2500/2500 [==============================] - 2167s 867ms/step - loss: 1.4951 - balanced_accuracy: 0.3178 - val_loss: 1.4725 - val_balanced_accuracy: 0.3470
Epoch 5/5
2500/2500 [==============================] - 2190s 876ms/step - loss: 1.3492 - balanced_accuracy: 0.4102 - val_loss: 1.2254 - val_balanced_accuracy: 0.5030


In [ ]:
predicted_raw = model.predict({'input_ids':x_test['input_ids'],'attention_mask':x_test['attention_mask']})
predicted_raw[0]

In [ ]:
y_predicted = np.argmax(predicted_raw, axis = 1)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_true, t_test))